### 导入库

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### 导入excel

In [16]:
with pd.ExcelFile(r'D:\小鸡理财\百度云同步盘\小鸡理财\数据报告\网站运营报告\8月运营报告\8月运营报告数据.xlsx') as xlsx:
    df_tz = pd.read_excel(xlsx,'投资')
    df_zc = pd.read_excel(xlsx,'注册')
    df_yj = pd.read_excel(xlsx,'佣金')
    df_vip = pd.read_excel(xlsx,'VIP变动')
    df_quan = pd.read_excel(xlsx,'券')
    df_zht = pd.read_excel(xlsx,'智慧投')

## 2页

- 成交额

In [3]:
cheng_jiao_sum = df_tz['总投资额'].sum()
print('8月成交额：',cheng_jiao_sum)

8月成交额： 51862600.0


- 为用户赚取收益

In [30]:
df_tz['加息'] = df_tz['加息利率'].str.rstrip('%').astype('float') / 100.0

In [32]:
df_tz['加息标'] = df_tz['加息'] * df_tz['总投资额'] * df_tz['投资期限'] / 360

In [34]:
df_tz['加息标'].sum()

365701.35833333334

In [38]:
diyong_quan_sum = df_quan[df_quan['大类']=='抵用券'].金额.sum()
xianjin_quan_sum = df_quan[df_quan['大类']=='现金券'].金额.sum()
jiaxi_quan_sum = df_quan[df_quan['大类']=='加息券'].成本.sum()

jiaxibiao_sum = df_tz['加息标'].sum()
tz_shouyi_sum = df_tz['预期收益'].sum()
yj_sum = df_yj['佣金金额'].sum()

print('投资{0} + 佣金{1} + 抵用券{2} + 现金券{3} + 加息券{4} + 加息标{5:.2f}= {6:.2f}'
      .format(tz_shouyi_sum,
              yj_sum,
              diyong_quan_sum,
              xianjin_quan_sum,
              jiaxi_quan_sum,
              jiaxibiao_sum,
              tz_shouyi_sum+yj_sum+diyong_quan_sum+xianjin_quan_sum+jiaxi_quan_sum+jiaxibiao_sum))

投资3737268.43 + 佣金226350.36 + 抵用券54077 + 现金券1862 + 加息券279866.08 + 加息标365701.36= 4665125.23


- 平台累计总交易额

In [6]:
last = input('上月累计成交额')

print('\n平台累计总交易额：上月{0} + 本月{1} = {2}'\
      .format(float(last),cheng_jiao_sum,float(last) + cheng_jiao_sum))

上月累计成交额2393795084.03

平台累计总交易额：上月2393795084.03 + 本月51862600.0 = 2445657684.03


## 3页

- 新增用户数

In [7]:
fresh_users = df_zc['用户名'].count()

print('7月新增用户数：',fresh_users)

7月新增用户数： 2415


- 累计注册用户数

In [8]:
last_users = input('上月累计注册用户数')

print('\n平台累计注册用户数：上月{0} + 本月{1} = {2}'\
      .format(int(last_users),fresh_users,int(last_users) + fresh_users))

上月累计注册用户数237809

平台累计注册用户数：上月237809 + 本月2415 = 240224


## 4页

In [9]:
df_tz['投资来源'].unique()

array(['苹果端', '安卓端', '微信端', 'Web端', '手机H5'], dtype=object)

- 移动端投资额

In [10]:
tz_yidong_sum = df_tz[df_tz['投资来源'].isin(['安卓端','苹果端', '微信端'])].总投资额.sum()

print('移动端投资额：',tz_yidong_sum)

移动端投资额： 51268400.0


- 比例

In [11]:
gp = df_tz.groupby('投资来源')['总投资额'].sum()

temp = pd.DataFrame(gp)

temp['total'] = cheng_jiao_sum

temp['percent'] = temp['总投资额'] / temp['total']

print('APP {0:.2%} , 微信 {1:.2%} , PC {2:.2%}'.format(
    temp.loc['安卓端','percent'] + temp.loc['苹果端','percent'],
    temp.loc['微信端','percent'],
    temp.loc['Web端','percent']))

APP 94.64% , 微信 4.22% , PC 1.14%


In [12]:
#方法二：
dt = {'安卓端':'APP','苹果端':'APP','微信端':'微信','Web端':'PC'}

df_tz['group'] = df_tz['投资来源'].map(dt)
grouped = df_tz.groupby('group')['总投资额'].sum()

gp_pct = grouped.apply(lambda x : 100 * x / cheng_jiao_sum)
gp_pct

group
APP    94.635055
PC      1.140128
微信      4.219225
Name: 总投资额, dtype: float64

## 5页

- 月度土豪奖

In [20]:
gp_tz_1 = df_zht.groupby('手机号码')['总投资金额'].sum().sort_values(ascending=False)

phone = gp_tz_1.index[0]
num = gp_tz_1.iloc[0]

print('月度土豪奖:{0} | {1:.0f}万'.format(phone,num/10000))

月度土豪奖:18066446644 | 319万


- 月度勤恳奖

In [21]:
gp_tz_2 = df_zht.groupby('手机号码')['总投资金额'].count().sort_values(ascending=False)

phone = gp_tz_2.index[0]
num = gp_tz_2.iloc[0]

print('月度勤恳奖:{0} | {1}次'.format(phone,num))

月度勤恳奖:18066446644 | 82次


- 月度收获奖

In [22]:
df_zht['profit'] = df_zht['预期利息']+df_zht['抵用券成本']+df_zht['加息券成本']+df_zht['加息标成本']

gp_tz_3 = df_zht.groupby('手机号码')['profit'].sum().sort_values(ascending=False)

phone = gp_tz_3.index[0]
num = gp_tz_3.iloc[0]

print('月度收获奖:{0} | {1:.0f}万'.format(phone,num/10000))

月度收获奖:18066446644 | 53万


- 月度人气奖

In [23]:
gp_zc_1 = df_zc.groupby('邀请人')['用户名'].count().sort_values(ascending=False)

phone = gp_zc_1.index[0]
num = gp_zc_1.iloc[0]

print('月度人气奖:{0:.0f} | {1}人'.format(phone,num))

月度人气奖:13705740830 | 120人


## 6页

- “壕”气冲天

In [24]:
gp_tz_4 = df_tz.groupby('投资时会员类型')['总投资额'].sum()
# print(gp_tz_4)

gp_tz_5 = df_tz.groupby('投资时会员类型')['投资用户'].nunique()
# print(gp_tz_5)

gp_tz_6 = pd.concat([gp_tz_4, gp_tz_5], axis=1)
gp_tz_6['avg'] = gp_tz_6['总投资额'] / gp_tz_6['投资用户']
gp_tz_6.sort_values(by = 'avg',ascending = False,inplace = True)

vip = gp_tz_6.index[0]
num = gp_tz_6.iloc[0,2]

print('“壕”气冲天:{0} | {1:.0f}万'.format(vip,num/10000))

“壕”气冲天:VIP7 | 163万


- “勇”争先锋

In [25]:
gp_tz_4.sort_values(ascending = False,inplace = True)

vip = gp_tz_4.index[0]
num = gp_tz_4.iloc[0]

print('“勇”争先锋:{0} | {1:.0f}万'.format(vip,num/10000))

“勇”争先锋:VIP2 | 1218万


- VIP用户升级情况

In [26]:
df_vip_up = df_vip[df_vip['变动原因'] == '升级']
# df_vip_up['变动原因'].unique()

df_vip_up.sort_values(by = '变动后等级',ascending = False,inplace = True)
df_vip_up.drop_duplicates(subset=['用户名'],inplace = True)

# df_vip_up.info()
gp_vip = df_vip_up.groupby('变动后等级')['用户名'].count()
gp_vip.index.name = 'VIP 升级人数' 

print('{0}位用户的VIP等级提升，其中：\n'.format(gp_vip.sum()))
gp_vip

304位用户的VIP等级提升，其中：



C:\Users\Xiaoji\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Xiaoji\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


VIP 升级人数
1    138
2     93
3     40
4     16
5     13
6      3
7      1
Name: 用户名, dtype: int64